In [ ]:
%load_ext aiida
%aiida

In [ ]:
from aiidalab_atmospec_workchain import OrcaWignerSpectrumWorkChain
from aiida.engine import WorkChain, calcfunction
from aiida.engine import submit, run, append_, ToContext, if_
from aiida.engine import run_get_node, run_get_pk

StructureData = DataFactory("core.structure")
Dict = DataFactory("core.dict")
TrajectoryData = DataFactory("core.array.trajectory")

# Test more than one conformer

In [ ]:
builder = AtmospecWorkChain.get_builder()
old_workchain = load_node(pk=226)
builder.structure = old_workchain.inputs.structure
for input in old_workchain.inputs:
    if input != 'structure':
        builder[input] = old_workchain.inputs[input]
        
# Patch the inputs to reduct comp cost
builder.nwigner = 2

params = builder.opt.orca.parameters.get_dict()
params['input_keywords'] = ['sto-3g', 'pbe', 'Opt', 'AnFreq']
builder.opt.orca.parameters = Dict(dict=params)

params = builder.exc.orca.parameters.get_dict()
params['input_keywords'] = ['sto-3g', 'pbe']
builder.exc.orca.parameters = Dict(dict=params)

# Not sure why this is not already included
builder.opt.orca.metadata.options.resources = {'tot_num_mpiprocs': 1}
builder.exc.orca.metadata.options.resources = {'tot_num_mpiprocs': 1}
builder.opt.clean_workdir = Bool(True)
builder.exc.clean_workdir = Bool(True)
builder

In [ ]:
output = run(builder)
output

In [ ]:
from aiidalab_atmospec_workchain import structures_to_trajectory
from aiida.engine import WorkChain
OrcaBaseWorkChain = WorkflowFactory("orca.base")
from aiida.engine import append_, ToContext

StructureData = DataFactory("core.structure")
TrajectoryData = DataFactory("core.array.trajectory")
Array = DataFactory("core.array")
Code = DataFactory("core.code.installed")

class RobustOptimizationWorkChain(WorkChain):
    """Molecular geometry optimization WorkChain that automatically
    detects imaginary frequencies and restarts the optimization
    until a true minimum is found.
    """

    def _build_process_label(self):
        return "Robust Optimization"

    @classmethod
    def define(cls, spec):
        super().define(spec)

        spec.expose_inputs(OrcaBaseWorkChain, exclude=["orca.structure", "orca.code"])
        spec.input("structure", valid_type=StructureData)
        spec.input("code", valid_type=Code)

        spec.outline(
            cls.optimize,
            cls.inspect_optimization,
            cls.results,
        )

        spec.expose_outputs(OrcaBaseWorkChain)

        spec.exit_code(
            401,
            "ERROR_OPTIMIZATION_FAILED",
            "optimization encountered unspecified error",
        )

    def optimize(self):
        """Optimize molecular geometry"""
        inputs = self.exposed_inputs(OrcaBaseWorkChain, agglomerate=False)
        inputs.orca.structure = self.inputs.structure
        inputs.orca.code = self.inputs.code

        calc_opt = self.submit(OrcaBaseWorkChain, **inputs)
        calc_opt.label = "robust-optimization"
        return ToContext(calc_opt=calc_opt)

    def inspect_optimization(self):
        """Check whether optimization succeeded"""
        if not self.ctx.calc_opt.is_finished_ok:
            return self.exit_codes.ERROR_OPTIMIZATION_FAILED

    def results(self):
        self.out_many(self.exposed_outputs(self.ctx.calc_opt, OrcaBaseWorkChain))

In [ ]:
builder = RobustOptimizationWorkChain.get_builder()

In [ ]:
#from aiida.plugins import load_code
Dict = DataFactory('core.dict')
old_workchain = load_node(pk=223)
builder.structure = old_workchain.inputs.structure.get_structure(index=0)
params = {'charge': 0, 'multiplicity': 1, 'extra_input_keywords': []}
params['input_keywords'] = ['sto-3g', 'pbe', 'Opt', 'AnFreq']
builder.orca.parameters = Dict(params)
builder.code = load_code('orca@localhost')
resources = {
    'num_mpiprocs_per_machine': 1,
    'num_machines': 1,
    'tot_num_mpiprocs': 1
}
builder.orca.metadata.options.resources = resources
#builder.metadata = resources
#builder

In [ ]:
from aiida.engine import submit, run
outputs, proc = run_get_node(RobustOptimizationWorkChain, **builder)

In [ ]:
class ConformerOptimizationWorkChain(WorkChain):
    """Top-level workchain for optimization of molecules.

    Essentially, this is a thin wrapper workchain around RobustOptimizationWorkChain
    to support optimization of multiple conformers in parallel.
    """

    def _build_process_label(self):
        return "Conformer Optimization Workflow"

    @classmethod
    def define(cls, spec):
        super().define(spec)
        spec.expose_inputs(RobustOptimizationWorkChain, exclude=["structure"])
        spec.input("structure", valid_type=(StructureData, TrajectoryData))

        spec.output(
            "relaxed_structures",
            valid_type=TrajectoryData,
            required=True,
            help="Minimized structures of all conformers",
        )

        spec.outline(
            cls.launch_conformer_optimization,
            cls.inspect_conformer_optimization,
            cls.collect_optimized_conformers,
        )

        # Very generic error now
        spec.exit_code(300, "CONFORMER_ERROR", "Conformer optimization failed")

    def launch_conformer_optimization(self):
        inputs = self.exposed_inputs(RobustOptimizationWorkChain, agglomerate=False)
        # Single conformer
        # TODO: Test this!
        if isinstance(self.inputs.structure, StructureData):
            self.report("Launching Optimization for 1 conformer")
            inputs.structure = self.inputs.structure
            return ToContext(confs=self.submit(RobustOptimizationWorkChain, **inputs))

        nconf = len(self.inputs.structure.get_stepids())
        self.report(
            f"Launching optimization for {nconf} conformers"
        )
        for conf_id in self.inputs.structure.get_stepids():
            inputs.structure = self.inputs.structure.get_step_structure(conf_id)
            workflow = self.submit(RobustOptimizationWorkChain, **inputs)
            workflow.label = f"optimize-conformer-{conf_id}"
            self.to_context(confs=append_(workflow))

    def inspect_conformer_optimization(self):
        """Check whether all optimizations succeeded"""
        # TODO: Specialize errors. Can we expose errors from child workflows?
        if isinstance(self.inputs.structure, StructureData):
            if not self.ctx.confs.is_finished_ok:
                return self.exit_codes.CONFORMER_ERROR
            return
        for wc in self.ctx.confs:
            if not wc.is_finished_ok:
                return self.exit_codes.CONFORMER_ERROR


    def collect_optimized_conformers(self):
        """Combine all optimized geometries into single TrajectoryData"""
        # TODO: Include energies in TrajectoryData for optimized structures
        # TODO: Calculate Boltzmann weights and append them to TrajectoryData
        if isinstance(self.inputs.structure, StructureData):
            relaxed_structures = {"struct_0": self.ctx.confs.outputs.relaxed_structure}
        else:
            relaxed_structures = {
                f"struct_{i}": wc.outputs.relaxed_structure
                for i, wc in enumerate(self.ctx.confs)
            }
        # TODO: We should preserve the stepids from the input TrajectoryData
        trajectory = structures_to_trajectory(**relaxed_structures)
        self.out("relaxed_structures", trajectory)

In [ ]:
builder = ConformerOptimizationWorkChain.get_builder()
Dict = DataFactory('core.dict')
old_workchain = load_node(pk=223)
builder.structure = old_workchain.inputs.structure.get_structure(index=0)
params = {'charge': 0, 'multiplicity': 1, 'extra_input_keywords': []}
params['input_keywords'] = ['sto-3g', 'pbe', 'Opt', 'AnFreq']
builder.orca.parameters = Dict(params)
builder.code = load_code('orca@localhost')
resources = {
    'num_mpiprocs_per_machine': 1,
    'num_machines': 1,
    'tot_num_mpiprocs': 1
}
builder.orca.metadata.options.resources = resources
#builder.metadata = resources
#builder

In [ ]:
from aiida.engine import run_get_node
outputs, proc = run_get_node(ConformerOptimizationWorkChain, **builder)